In [19]:
import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import *
from tkinter import messagebox
import tkinter.simpledialog
import matplotlib
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
import pandas_datareader as pdr
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectFromModel
import statsmodels.formula.api as smf
from pykalman import KalmanFilter
from sklearn.decomposition import PCA
from scipy.optimize import minimize

In [20]:
def pop_up_box(asset_list):
    import tkinter as tk
    def ok():
        nonlocal risk_budget
        for i in range(len(asset_list)):
            risk_budget.append(globals()['weight'+str(i)].get())
        if np.sum(risk_budget) != 1:
            messagebox.showerror("Error", "Please make sure the sum is one")
        else:
            messagebox.showinfo("Info", "Enter budget successfully, please close the window")


    def cancel():
        nonlocal risk_budget
        risk_budget = []

    risk_budget = []
    root = tk.Toplevel() 
    root.title('Setting risk budget')
    #root.geometry('500x200')    

    for i in range(len(asset_list)):
        locals()['row' + str(i)] = tk.Frame(root)
        locals()['row' + str(i)].pack(fill="x")
        tk.Label(locals()['row' + str(i)], text='{}：'.format(asset_list[i]), width=8).pack(side=tk.LEFT)
        globals()['weight'+str(i)] = tk.DoubleVar()
        tk.Entry(locals()['row' + str(i)], textvariable=globals()['weight'+str(i)], width=20).pack(side=tk.LEFT)

    row_last = tk.Frame(root)
    row_last.pack(fill="x")
    tk.Button(row_last, text="Cancel", command=cancel).pack(side=tk.RIGHT)
    tk.Button(row_last, text="Enter", command=ok).pack(side=tk.RIGHT)
    con = tk.Label(root, text='Please enter the risk budget for each stock, make sure it sums up to 1').pack()
    #con.place(x=10,y=180)
    #root.mainloop()
    return risk_budget

In [21]:
class PortfolioAnalyzer:
    def __init__(self):
        import pandas_datareader as pdr
        self.pool = pd.DataFrame()
        self.asset = []
        self.startDate='2019-01-01'
        self.close=pd.DataFrame()
        self.ret= pd.DataFrame()
        self.market=pd.DataFrame()
        self.factor = pd.DataFrame()
        self.weight = pd.DataFrame()
        self.cov = 0
        self.sigma = 0
        self.NAV = pd.DataFrame()
        self.returns=pd.DataFrame()
        self.fd=pd.DataFrame()
        self.lassof=[]
        self.stepwisef=[]
        self.pca=[]
        self.risk_target=[]
        
    def initialization(self):
        import pandas_datareader as pdr
        self.pool=pd.read_csv('input.csv')
        self.asset = self.pool.ticker.values.tolist()
        self.startDate = tkinter.simpledialog.askstring("Start Date: " , "Enter start date, eg. '2015-01-01': " ,initialvalue='2016-01-01')
        if self.startDate is None:
            return
        self.close = pdr.DataReader(self.asset, data_source='yahoo', start=self.startDate)['Adj Close']
        #self.factor = pdr.DataReader('F-F_Research_Data_Factors_daily', 'famafrench',start=self.startDate)[0][['Mkt-RF','SMB','HML']]
        self.returns = self.close/self.close.shift(1) - 1
        self.cov = np.matrix(self.close.cov())
        self.weight = self.pool['weighted']
        self.sigma = np.array(self.close.std())
        #self.NAV = (self.close*np.tile(self.weight,(self.close.shape[0],1))).sum(axis=1)
        #self.factor = pd.read_csv('factors.csv')
        self.market = pdr.DataReader('SPY', data_source='yahoo', start=self.startDate)['Adj Close']
        self.market = self.market/self.market.values[0]
        messagebox.showinfo("MessageBox", "Initialization Finished! Don't forget to select one of buttons 3-6 to compute fund returns and NAV.")
    
    def initialization_method2(self):
        import pandas_datareader as pdr
        fund = pd.read_csv('input_method2.csv',index_col=0)
        self.ret=fund[['Returns']]
        self.NAV=fund[['NAV']]
        self.startDate=fund.index.values[0]
        self.market = pdr.DataReader('SPY', data_source='yahoo', start=self.startDate)['Adj Close']
        self.market = self.market/self.market.values[0]
        messagebox.showinfo("MessageBox", "Initialization Finished!")
        con = tk.Label(window, text='【Current State】Upload Fund NAV')
        con.place(x=10,y=180)
        
    def analyst(self):
        if self.pool.empty:
            messagebox.showinfo("MessageBox", "Please finish initialization by button[1]!")
            return
        Analyst = tkinter.simpledialog.askstring("Analyst's Name: " , "Enter Analyst's name, eg. 'Amy','Bob','John':" )
        if Analyst is None:
            return
        con = tk.Label(window, text='【Current State】Selected by Analyst: '+Analyst)
        con.place(x=10,y=180)        
        if self.pool[self.pool['analyst']==Analyst].empty:
            messagebox.showinfo("MessageBox", "Your input is not in pool, please check!")
            return            
        totalweighted=self.pool[self.pool['analyst']==Analyst].weighted.sum()
        for i in range(0,len(self.pool[self.pool['analyst']==Analyst])):
            if i==0:
                Weighted=self.returns[self.pool[self.pool['analyst']==Analyst].iloc[i,0]]*self.pool[self.pool['analyst']==Analyst].iloc[i,-1]/totalweighted
            else:
                Weighted=Weighted+self.returns[self.pool[self.pool['analyst']==Analyst].iloc[i,0]]*self.pool[self.pool['analyst']==Analyst].iloc[i,-1]/totalweighted
                
        top=Toplevel()
        top.title('Plot')
        self.NAV=pd.DataFrame(Weighted.cumsum()+1,columns=['NAV'],index=self.close.index)
        self.ret=pd.DataFrame(Weighted,columns=['Returns'],index=self.close.index)
        f =Figure(figsize=(5,4), dpi=100)
        a = f.add_subplot(111)
        self.NAV.plot(ax=a)
        a.set_title(Analyst+' Fund NAV')
        canvas =FigureCanvasTkAgg(f, master=top)
        #canvas.show()
        canvas.get_tk_widget().pack()
        
        
    def sector(self):
        if self.pool.empty:
            messagebox.showinfo("MessageBox", "Please finish initialization by button[1]!")
            return
        
        Sector = tkinter.simpledialog.askstring("Sector's Name: " , "Enter Sector's name, eg. 'Technology','Finance','Health':")
        if Sector is None:
            return
        con = tk.Label(window, text='【Current State】Selected by Sector: '+Sector)
        con.place(x=10,y=180)
        if self.pool[self.pool['sector']==Sector].empty:
            messagebox.showinfo("MessageBox", "Your input is not in pool, please check!")
            return              
        totalweighted=self.pool[self.pool['sector']==Sector].weighted.sum()
        for i in range(0,len(self.pool[self.pool['sector']==Sector])):
            if i==0:
                Weighted=self.returns[self.pool[self.pool['sector']==Sector].iloc[i,0]]*self.pool[self.pool['sector']==Sector].iloc[i,-1]/totalweighted
            else:
                Weighted=Weighted+self.returns[self.pool[self.pool['sector']==Sector].iloc[i,0]]*self.pool[self.pool['sector']==Sector].iloc[i,-1]/totalweighted

        top=Toplevel()
        top.title('Plot')
        self.NAV=pd.DataFrame(Weighted.cumsum()+1,columns=['NAV'],index=self.close.index)
        self.ret=pd.DataFrame(Weighted,columns=['Returns'],index=self.close.index)
        f =Figure(figsize=(5,4), dpi=100)
        a = f.add_subplot(111)
        self.NAV.plot(ax=a)
        a.set_title(Sector+' Fund NAV')
        canvas =FigureCanvasTkAgg(f, master=top)
        #canvas.show()
        canvas.get_tk_widget().pack()


        
    def theme(self):
        if self.pool.empty:
            messagebox.showinfo("MessageBox", "Please finish initialization by button[1]!")
            return
        Theme = tkinter.simpledialog.askstring("Theme's Name: " , "Enter theme's name, eg. 'growing','leading':" )
        if Theme is None:
            return
        con = tk.Label(window, text='【Current State】Selected by Theme: '+Theme)
        con.place(x=10,y=180)
        if self.pool[self.pool['theme']==Theme].empty:
            messagebox.showinfo("MessageBox", "Your input is not in pool, please check!")
            return                
        totalweighted=self.pool[self.pool['theme']==Theme].weighted.sum()
        for i in range(0,len(self.pool[self.pool['theme']==Theme])):
            if i==0:
                Weighted=self.returns[self.pool[self.pool['theme']==Theme].iloc[i,0]]*self.pool[self.pool['theme']==Theme].iloc[i,-1]/totalweighted
            else:
                Weighted=Weighted+self.returns[self.pool[self.pool['theme']==Theme].iloc[i,0]]*self.pool[self.pool['theme']==Theme].iloc[i,-1]/totalweighted

        top=Toplevel()
        top.title('Plot')
        self.NAV=pd.DataFrame(Weighted.cumsum()+1,columns=['NAV'],index=self.close.index)
        self.ret=pd.DataFrame(Weighted,columns=['Returns'],index=self.close.index)
        f =Figure(figsize=(5,4), dpi=100)
        a = f.add_subplot(111)
        self.NAV.plot(ax=a)
        a.set_title(Theme+' Fund NAV')
        canvas =FigureCanvasTkAgg(f, master=top)
        #canvas.show()
        canvas.get_tk_widget().pack()
        

    
    def allstock(self):
        if self.pool.empty:
            messagebox.showinfo("MessageBox", "Please finish initialization by button[1]!")
            return
        con = tk.Label(window, text='【Current State】Selected: All in pools')
        con.place(x=10,y=180)
        
        totalweighted=self.pool[abs(self.pool['weighted'])<1].weighted.sum()
        for i in range(0,len(self.pool[abs(self.pool['weighted'])<1])):
            if i==0:
                Weighted=self.returns[self.pool[abs(self.pool['weighted']<=1)].iloc[i,0]]*self.pool[abs(self.pool['weighted']<=1)].iloc[i,-1]/totalweighted
            else:
                Weighted=Weighted+self.returns[self.pool[abs(self.pool['weighted']<=1)].iloc[i,0]]*self.pool[abs(self.pool['weighted']<=1)].iloc[i,-1]/totalweighted

        top=Toplevel()
        top.title('Plot')
        self.NAV=pd.DataFrame(Weighted.cumsum()+1,columns=['NAV'],index=self.close.index)
        self.ret=pd.DataFrame(Weighted,columns=['Returns'],index=self.close.index)
        f =Figure(figsize=(5,4), dpi=100)
        a = f.add_subplot(111)
        self.NAV.plot(ax=a)
        a.set_title('All in pool Fund NAV')
        canvas =FigureCanvasTkAgg(f, master=top)
        #canvas.show()
        canvas.get_tk_widget().pack()
        

        
    def calculate_portfolio_var(self,weight,cov):
        w = np.matrix(weight)
        return (w*cov*w.T)[0,0]
                
    def market_neutral(self):
        #tot_df = pd.DataFrame()
        #tot_df['market'] = self.market
        #tot_df['portfolio'] = self.NAV
        if self.NAV.empty:
            messagebox.showinfo("MessageBox", "Please finish initialization by button [1] or [2]!")
            return
        tot_df=self.NAV.merge(self.market,left_index=True,right_index=True)
        tot_df=tot_df.dropna()
        tot_df.columns=['portfolio','market']
        from sklearn.linear_model import LinearRegression
        lr = LinearRegression()
        lr.fit(np.array(tot_df['market']).reshape(-1, 1),tot_df['portfolio'])
        alpha = tot_df['portfolio'] - lr.predict(np.array(tot_df['market']).reshape(-1, 1))
        alpha=pd.DataFrame(alpha,index=tot_df.index)
        top=Toplevel()
        top.title('Plot')
        f =Figure(figsize=(5,4), dpi=100)
        a = f.add_subplot(111)
        alpha.plot(ax=a,legend=False)
        canvas =FigureCanvasTkAgg(f, master=top)
        a.set_title('Market Neutral')
        #canvas.show()
        canvas.get_tk_widget().pack()

    def plot_performance(self):
        if self.NAV.empty:
            messagebox.showinfo("MessageBox", "Please finish initialization by button [1] or [2]!")
            return
        windowt = tk.Toplevel()
        tot_df=self.NAV.merge(self.market,left_index=True,right_index=True)
        tot_df=tot_df.dropna()
        tot_df.columns=['Fund NAV','Market NAV']
        
        tk.Label(windowt, text='Fund performance summary:',font=('Arial',13,'bold')).pack()
        fig = Figure(figsize=(5,4), dpi=100)
        ax = fig.add_subplot(111)
        tot_df.plot( ax=ax)
        canvas = FigureCanvasTkAgg(fig, master=windowt)
        canvas.get_tk_widget().pack()
        
        tk.Label(windowt, text='Annual expected Return:',font=('Arial',13,'bold')).pack()
        tk.Label(windowt, text=round(self.ret.mean()[0]*252,4)).pack()
        tk.Label(windowt, text='Annual Std:',font=('Arial',13,'bold')).pack()
        tk.Label(windowt, text=round(self.ret.std()[0]*np.sqrt(252),4)).pack()
                
        
    def upload_factor(self):
        if self.NAV.empty:
            messagebox.showinfo("MessageBox", "Please finish initialization!")
            return
        self.factor=pd.read_csv('factor.csv',index_col=0)
        self.fd=self.ret.merge(self.factor, left_index=True, right_index=True)
        messagebox.showinfo("MessageBox", 'Finished upload factors data!')
    
    def download_factor(self):
        if self.NAV.empty:
            messagebox.showinfo("MessageBox", "Please finish initialization!")
            return
        self.factor=pdr.DataReader('F-F_Research_Data_Factors_daily', 'famafrench',start=self.startDate)[0][['Mkt-RF','SMB','HML']]
        self.factor.columns=['Mkt','SMB','HML']
        self.fd=self.ret.merge(self.factor, left_index=True, right_index=True)
        messagebox.showinfo("MessageBox", 'Finished download factors data!')

    def LASSO_selected(self):
        if self.factor.empty:
            messagebox.showinfo("MessageBox", "Please upload or download factors data by button [13] or [14]!")
            return
        df=self.fd
        df=df.dropna()
        target='Returns'
        alphai=tkinter.simpledialog.askstring("LASSO Alpha: " , "Enter alpha, eg. '0.001':" )
        if alphai is None:
            return
        alphai=float(alphai)
        from sklearn.linear_model import Lasso
        from sklearn.metrics import r2_score
        from sklearn.metrics import mean_squared_error
        from sklearn.feature_selection import SelectFromModel
        allvar=list(df)
        allvar.remove(target)
        X_train=df[allvar]
        y_train=df[[target]]
        # normalize the variable before LASSO Regression.
        model_lasso = Lasso(alpha=alphai)
        model_lasso.fit(X_train, y_train) 
        pred_train_lasso= model_lasso.predict(X_train)
        coef=[round(i,10) for i in model_lasso.coef_]
        selvar=[allvar[i] for i,n in enumerate(coef) if n != 0]
        #print('Coef: ',coef)
        windowt = tk.Toplevel()
        #windowt.geometry("300x300")
        tk.Label(windowt, text='Selected Variables: ',font=('Arial',13,'bold')).pack()
        tk.Label(windowt, text=selvar).pack()
        tk.Label(windowt, text='RS: ',font=('Arial',13,'bold')).pack()
        tk.Label(windowt, text=round(r2_score(y_train, pred_train_lasso),2)).pack()
        tk.Label(windowt, text='MSE: ',font=('Arial',13,'bold')).pack()
        tk.Label(windowt, text=round(np.sqrt(mean_squared_error(y_train,pred_train_lasso)),2)).pack()
        self.lassof=selvar
        
    def forward_selected(self):
        if self.factor.empty:
            messagebox.showinfo("MessageBox", "Please upload or download factors data by button [13] or [14]!")
            return
        import statsmodels.formula.api as smf
        data=self.fd
        data=data.dropna()
        target='Returns'
        method='AIC'
        
        variate = set(data.columns) 
        variate.remove(target) 
        selected = []
        current_score, best_new_score = float('inf'), float('inf') 
        while variate:
            score_with_variate = [] 
            for candidate in variate:  
                formula = "{}~{}".format(target,"+".join(selected+[candidate]))  
                if method == 'AIC':
                    score = smf.ols(formula=formula, data=data).fit().aic 
                else:
                    score=smf.ols(formula=formula, data=data).fit().bic 
                score_with_variate.append((score, candidate))  
            score_with_variate.sort(reverse=True)  
            best_new_score, best_candidate = score_with_variate.pop()  
            if current_score > best_new_score:  
                variate.remove(best_candidate)
                selected.append(best_candidate)  
                current_score = best_new_score  
            else:
                break
        formula = "{}~{}".format(target,"+".join(selected))  
        model = smf.ols(formula=formula, data=data).fit()
        
        windowt= tk.Toplevel()
        #windowt.geometry("300x300")
        tk.Label(windowt, text='Selected Variables: ',font=('Arial',13,'bold')).pack()
        tk.Label(windowt, text=model.params.index[1:].tolist()).pack()        
        self.stepwisef=model.params.index[1:].tolist()
    
    def OLS_method(self):       
        if self.factor.empty:
            messagebox.showinfo("MessageBox", "Please upload or download factors data by button [13] or [14]!")
            return
        Xnamelist=tkinter.simpledialog.askstring("Factors list " , "Enter factors (list), eg. Mkt or Mkt,SMB,HML:" )
        if Xnamelist is None:
            return
        Xnamelist=Xnamelist.split(',')  
        df=self.fd
        df=df.dropna()
        Yname='Returns'
        
        a=Xnamelist
        b=list(self.factor)
        d=[1 for c in a if c not in b]
        if len(d)>0:
            messagebox.showinfo("MessageBox", "Your input factor name error, please check!")
            return
        
        from sklearn.linear_model import LinearRegression
        Y=df[[Yname]]
        X=df[Xnamelist]
        OLS = LinearRegression(fit_intercept=True)
        OLS.fit(X,Y)
        Predict = OLS.predict(X)
        beta=OLS.coef_
        alpha=OLS.intercept_ 
        score=OLS.score(X,Y)
        #Y['Expected Value']=Predict
        #Y.plot()
        windowt = tk.Toplevel()
        tk.Label(windowt, text='Selected factors: ',font=('Arial',13,'bold')).pack()
        tk.Label(windowt, text=Xnamelist).pack()
        tk.Label(windowt, text='Alpha: ',font=('Arial',13,'bold')).pack()
        tk.Label(windowt, text=alpha).pack()
        tk.Label(windowt, text='Beta: ',font=('Arial',13,'bold')).pack()
        tk.Label(windowt, text=beta[0]).pack()
        
    def factor_neutral(self):  
        if self.factor.empty:
            messagebox.showinfo("MessageBox", "Please upload or download factors data by button [13] or [14]!")
            return
        Xnamelist=tkinter.simpledialog.askstring("Factor Neutral " , "Enter factor, eg. SMB" )
        if Xnamelist is None:
            return
        Xnamelist=Xnamelist.split(',')  
        
        a=Xnamelist
        b=list(self.factor)
        d=[1 for c in a if c not in b]
        if len(d)>0:
            messagebox.showinfo("MessageBox", "Your input factor name error, please check!")
            return
        
        df=self.fd
        df=df.dropna()
        Yname='Returns'
        
        from sklearn.linear_model import LinearRegression
        Y=df[[Yname]]
        X=df[Xnamelist]
        OLS = LinearRegression(fit_intercept=True)
        OLS.fit(X,Y)
        Predict = OLS.predict(X)
        beta=OLS.coef_
        alpha=OLS.intercept_ 
        score=OLS.score(X,Y)
        plotd=pd.DataFrame(Predict,columns=['Expected Returns'],index=Y.index)
        plotd=Y.merge(plotd,left_index=True,right_index=True)
        
        windowt = tk.Toplevel()
        fig = Figure(figsize=(5,4), dpi=100)
        ax = fig.add_subplot(111)
        ax.set_title(Xnamelist[0]+' Factor Neutral')
        plotd.plot( ax=ax)
        canvas = FigureCanvasTkAgg(fig, master=windowt)
        canvas.get_tk_widget().pack()
        
    def download(self,result,fname):
        filen=fname+'.csv'
        result.to_csv(filen)
        messagebox.showinfo("MessageBox", "Finished download!")
        
    def KF_method(self):
        if self.factor.empty:
            messagebox.showinfo("MessageBox", "Please upload or download factors data by button [13] or [14]!")
            return
        Xnamelist=tkinter.simpledialog.askstring("Factors list " , "Enter factors (list), eg. Mkt or Mkt,SMB,HML:" )
        if Xnamelist is None:
            return
        Xnamelist=Xnamelist.split(',')  

        a=Xnamelist
        b=list(self.factor)
        d=[1 for c in a if c not in b]
        if len(d)>0:
            messagebox.showinfo("MessageBox", "Your input factor name error, please check!")
            return
        
        df=self.fd
        df=df.dropna()
        Yname='Returns'
        
        if (df.shape[0]<=100):
            messagebox.showinfo("MessageBox", 'Not enough data!')
            return 
        
        def dataframe_mul(par,X):
            dsum=par.iloc[:,0]
            for i in range(X.shape[1]):
                dsum=dsum+par.iloc[:,i+1]*X.iloc[:,i]
            return dsum

        from pykalman import KalmanFilter
        Xlen=len(Xnamelist)+1
        observation_matrices = np.c_[np.ones(100),df.iloc[:100,][Xnamelist].values]
        Shape = observation_matrices.shape
        observation_matrices = observation_matrices.reshape(Shape[0],1,Shape[1])

        kf = KalmanFilter(transition_matrices=np.array(np.eye(Xlen)), 
                          observation_matrices=observation_matrices)
        np.random.seed(0)
        kf.em(df.iloc[:100,][Yname])
        filter_mean,filter_cov = kf.filter(df.iloc[:100,][Yname].values)
        start_index = 100

        for i in range(start_index,len(df)):
            observation_matrix = np.append(np.ones(1),df[Xnamelist].values[i]).reshape(1,-1)
            observation = df[Yname].values[i]
            next_filter_mean,next_filter_cov = kf.filter_update(
                    filtered_state_mean = filter_mean[-1],
                    filtered_state_covariance = filter_cov[-1],
                    observation = observation,
                    observation_matrix = observation_matrix)
            filter_mean = np.vstack((filter_mean,next_filter_mean))
            filter_cov = np.vstack((filter_cov,next_filter_cov.reshape(1,Xlen,Xlen)))

        results=pd.DataFrame(filter_mean)
        results.columns=['intercept']+Xnamelist
        results.index=df.index
        Predict=dataframe_mul(results,df[Xnamelist])
        #dplot=df[[Yname]]
        #dplot['Expected Value']=Predict
        #dplot.plot()
        windowt = tk.Toplevel()
        tk.Label(windowt, text='Selected factors: ',font=('Arial',13,'bold')).pack()
        tk.Label(windowt, text=Xnamelist).pack()
        tk.Label(windowt, text='Last Alpha: ',font=('Arial',13,'bold')).pack()
        tk.Label(windowt, text=results.iloc[-1,0]).pack()
        tk.Label(windowt, text='Last Beta: ',font=('Arial',13,'bold')).pack()
        tk.Label(windowt, text=results.iloc[-1,1:].values).pack()
        kf1=tk.Button(windowt,text='Download Whole Alpha Beta',font=('Arial',12),width=30,height=2, 
                      command=lambda: pa.download(results,'KF_alpha_beta')).pack()

        
    def min_max(self):
        if self.pool.empty:
            messagebox.showinfo("MessageBox", "Please upload stock pool by button [1]!")
            return
        asset = self.asset
        df = self.close
        ret = df.pct_change()
        mean_ret = ret.mean()
        covMatrix = ret.cov()
        number_of_assets = len(asset)

        niter = 15000
        sim_res = np.zeros((4+len(asset)-1,niter))

        for i in range(niter):
            #Random weights
            wei = np.array(np.random.random(number_of_assets))
            #Constraint sum(Weights) = 1
            wei /= np.sum(wei)

            #Annualized return
            PRet = np.sum(mean_ret * wei)*252
            #Annualized standard deviation
            PStdDev = np.sqrt(np.dot(wei.T,np.dot(covMatrix, wei))) * np.sqrt(252)

            sim_res[0,i] = PRet
            sim_res[1,i] = PStdDev

            #Sharpe ratio
            sim_res[2,i] = sim_res[0,i] / sim_res[1,i]

            for j in range(len(wei)):
                sim_res[j+3,i] = wei[j]
        
        #Column names
        columns = ['Return','StDev','Sharpe_ratio']
        for i in asset:
            columns.append(i)

        sim_frame = pd.DataFrame(sim_res.T,columns=columns)

        max_sharpe = sim_frame.iloc[sim_frame['Sharpe_ratio'].idxmax()]
        min_std = sim_frame.iloc[sim_frame['StDev'].idxmin()]
        max_ret = sim_frame.iloc[sim_frame['Return'].idxmax()]
        windowt = tk.Toplevel()
        #windowt.geometry("300x300")
        tk.Label(windowt, text='Maximum Sharpe: ',font=('Arial',13,'bold')).pack()
        tk.Label(windowt, text=max_sharpe).pack()
        tk.Label(windowt, text='Minimum Standard Deviation: ',font=('Arial',13,'bold')).pack()
        tk.Label(windowt, text=min_std).pack()
        tk.Label(windowt, text='MAximum Return: ',font=('Arial',13,'bold')).pack()
        tk.Label(windowt, text=max_ret).pack()

        #return {'max_sharpe':max_sharpe, 'min_std':min_std, 'max_ret':max_ret}

    def factor_timing(self):
        if self.factor.empty:
            messagebox.showinfo("MessageBox", "Please upload or download factors data by button [13] or [14]!")
            return
        data=self.fd.dropna()
        returns=data['Returns']
        fac=tkinter.simpledialog.askstring("Factors name" , "Enter factor name, eg. 'SMB'" )
        if fac is None:
            return
        a=[fac]
        b=list(self.factor)
        d=[1 for c in a if c not in b]
        if len(d)>0:
            messagebox.showinfo("MessageBox", "Your input factor name error, please check!")
            return
        target=tkinter.simpledialog.askstring("Target Values" , "Enter target value, eg. 0" )
        if target is None:
            return
        target=float(target)
        ttype=tkinter.simpledialog.askstring("Type" , "Enter type, 'large' or 'small'" )
        if ttype is None:
            return
        factor=data[fac]
        
        if (ttype=='large'):
            factor=factor.values
            signal=np.zeros(len(factor))
            signal[np.where(factor>target)]=1
            pnl=(returns.values+1)*signal
            pnl[pnl==0]=1
            pnl[np.isnan(pnl)]=1
            pnl=pnl.cumprod()
        elif (ttype=='small'):
            factor=factor.values
            signal=np.zeros(len(factor))
            signal[np.where(factor<target)]=1
            pnl=(returns.values+1)*signal
            pnl[pnl==0]=1
            pnl[np.isnan(pnl)]=1
            pnl=pnl.cumprod()
        else:
            factor=factor.values
            signal=np.zeros(len(factor))
            signal[np.where(factor==target)]=1
            pnl=(returns.values+1)*signal
            pnl[pnl==0]=1
            pnl[np.isnan(pnl)]=1
            pnl=pnl.cumprod()
        
        pnl=pd.DataFrame(pnl,columns=['NAV'],index=data.index.values)        
        windowt = tk.Toplevel()
        tk.Label(windowt, text='Factor Timing: NAV Curve',font=('Arial',13,'bold')).pack()
        fig = Figure(figsize=(5,4), dpi=100)
        ax = fig.add_subplot(111)
        ax.set_title('Factors: ' +fac+' Type: '+ttype+' Target: '+str(target))
        pnl.plot( ax=ax)
        canvas = FigureCanvasTkAgg(fig, master=windowt)
        canvas.get_tk_widget().pack()

    def PCA(self):
        if self.factor.empty:
            messagebox.showinfo("MessageBox", "Please upload or download factors data by button [13] or [14]!")
            return
        from sklearn.decomposition import PCA
        pca = PCA(n_components='mle',svd_solver='full')
        pca.fit(self.factor)
        self.pca=pd.DataFrame(pca.explained_variance_ratio_)
        self.pca.columns=['Contribution']
        indexer=[]
        for i in range(1,len(self.pca)+1):
            indexer.append('factor '+str(i))
        self.pca.index=indexer
        windowt = tk.Toplevel()
        tk.Label(windowt, text='Explained variance for each factor: ',font=('Arial',13,'bold')).pack()
        tk.Label(windowt, text=self.pca.iloc[:,0]).pack()

        
    def Risk_budget(self,weight,cov,plot = True):
        w = np.matrix(weight)
        sigma = np.sqrt(self.calculate_portfolio_var(weight,cov))
        # Marginal Risk Contribution
        MRC = cov*w.T
        # Risk Contribution
        self.RC = np.multiply(MRC,w.T).reshape(1,-1)/sigma
        if plot == True:
            top=Toplevel()
            top.title('Plot')
            index = np.arange(len(self.asset))
            f =Figure(figsize=(5,4), dpi=100)
            a = f.add_subplot(111)
            a.bar(index,np.array(self.RC/self.RC.sum())[0])
            a.set_xlabel('Stocks')
            a.set_ylabel('Risk Contribution %')
            a.set_xticks(index)
            a.set_xticklabels(self.asset)
            canvas =FigureCanvasTkAgg(f, master=top)
            #canvas.show()
            canvas.get_tk_widget().pack()
            msg=Message(top,text='Risk Contribution')
            msg.pack()
        return self.RC
        
        
    def enter_risk_budget(self):
        if self.pool.empty:
            messagebox.showinfo("MessageBox", 'Please upload stock pool by button[1]!')
            return 
        self.risk_target = pop_up_box(self.asset)
        
    def risk_parity(self):
        if self.pool.empty:
            messagebox.showinfo("MessageBox", 'Please upload stock pool by button[1]!')
            return 
        risk_target = self.risk_target
        if risk_target == []:
            risk_target = [0.1]*10
        def objective(weight,cov,risk_target):
            portfolio_risk = np.sqrt(self.calculate_portfolio_var(weight,cov))
            rc = self.Risk_budget(weight,cov,plot=False)
            assets_risk_target = np.asmatrix(np.multiply(portfolio_risk, risk_target))
            return np.sum(np.square(rc - assets_risk_target))
        def total_weight_constraint(x):
            return np.sum(x)-1.0
        def long_only_constraint(x):
            return x
        cons = ({'type': 'eq', 'fun': total_weight_constraint},
                {'type': 'ineq', 'fun': long_only_constraint})
        res= minimize(objective, x0 = np.matrix(self.weight), args=(self.cov,risk_target), constraints=cons,options={'disp': False})
        self.new_weight = np.array(res.x)
        new_portfolio = (self.close*np.tile(self.new_weight,(self.close.shape[0],1))).sum(axis=1)
        new_portfolio = new_portfolio/new_portfolio[0]
        new_portfolio=pd.DataFrame(new_portfolio)

        
        top=Toplevel()
        top.title('Plot')
        f =Figure(figsize=(5,4), dpi=100)
        a = f.add_subplot(111)
        a.set_title('Backtest by tilting risk budget')
        new_portfolio.plot(ax=a,legend=False)
        canvas =FigureCanvasTkAgg(f, master=top)
        #canvas.show()
        canvas.get_tk_widget().pack()
        #msg=Message(top,text='Backtest by tilting risk budget')
        #msg.pack()
    
    def indicator(self):
        if self.NAV.empty:
            messagebox.showinfo("MessageBox", 'Please finish initialization by button[1] or [2]!')
            return 
        def max_drawdown(pricearry):
            maximums = pricearry.cummax()
            drawdowns = 1 - pricearry / maximums
            return np.max(drawdowns)
        def calmar_ratio(pricearry):
            'arry:price'
            returns = pricearry[-1]/pricearry[0] - 1
            annual_return=returns*252/pricearry.shape[0]
            return annual_return/max_drawdown(pricearry)
        def omega_ratio(returns, risk_free=0.00, required_return=0.0):
            annualization=252
            return_threshold = (1 + required_return) ** (1. / annualization) - 1
            returns_less_thresh = returns - risk_free - return_threshold
            numer = sum(returns_less_thresh[returns_less_thresh > 0.0])
            denom = -1.0 * sum(returns_less_thresh[returns_less_thresh < 0.0])
            if denom > 0.0:
                return numer / denom
            else:
                return np.nan
        def adjust_SR(returns,rf=0.0):
            annualreturn=returns*252/np.sqrt(returns.shape[0])
            SR=(annualreturn.mean()-rf)/annualreturn.std()
            S=annualreturn.skew()
            K=annualreturn.kurt()
            return SR*(1+(S/6)*SR-((K-3)/24)*(SR*SR))
        def cal_SR(returns,rf=0.0):
            annualreturn=returns*252/np.sqrt(returns.shape[0])
            SR=(annualreturn.mean()-rf)/annualreturn.std()
            return SR
        windowt = tk.Toplevel()
        #windowt.geometry("300x300")
        tk.Label(windowt, text='Annual Sharpe Ratio: ',font=('Arial',13,'bold')).pack()
        tk.Label(windowt, text=round(cal_SR(self.ret.dropna().values),4)).pack()
        tk.Label(windowt, text='Adjusted Sharpe Ratio: ',font=('Arial',13,'bold')).pack()
        tk.Label(windowt, text=round(adjust_SR(self.ret.dropna())[0],4)).pack()
        tk.Label(windowt, text='Maximum Draw Down: ',font=('Arial',13,'bold')).pack()
        tk.Label(windowt, text=round(max_drawdown(self.NAV.dropna())[0],4)).pack()
        tk.Label(windowt, text='Calmar Ratio: ',font=('Arial',13,'bold')).pack()
        tk.Label(windowt, text=round(calmar_ratio(self.NAV.dropna()['NAV']),4)).pack()
        tk.Label(windowt, text='Omega Ratio: ',font=('Arial',13,'bold')).pack()
        tk.Label(windowt, text=round(omega_ratio(self.ret.dropna().values),4)).pack()
        
        from pandas import Series
        from statsmodels.graphics.tsaplots import plot_acf
        from matplotlib import pyplot
        array=self.ret.dropna()

        fig = Figure(figsize=(5,4), dpi=100)
        ax = fig.add_subplot(111)
        plot_acf(array,ax=ax)
        #pyplot.show()
        canvas = FigureCanvasTkAgg(fig, master=windowt)
        canvas.get_tk_widget().pack()
        
    def draw_down(self):
        if self.NAV.empty:
            messagebox.showinfo("MessageBox", 'Please finish initialization by button[1] or [2]!')
            return 
        def get_max_drawdown(array):
            array = pd.DataFrame(array)
            cumsum = array.cummax()
            return np.max(cumsum-array)
        def drawdown(timeseries,window):
            output = np.full(timeseries.shape,np.nan)
            for n in range(window,timeseries.shape[0]):
                data = timeseries[n-window:n]
                output[n,:] = get_max_drawdown(data)
            return output
        windt=tkinter.simpledialog.askstring("Draw Down" , "Enter draw down window days, eg. 30" )
        if windt is None:
            return
        windt=int(windt)
        plotd=pd.DataFrame(drawdown(self.NAV.dropna(),windt))
        plotd.index=self.NAV.dropna().index
        
        windowt = tk.Toplevel()
        fig = Figure(figsize=(5,4), dpi=100)
        ax = fig.add_subplot(111)
        plotd.plot(ax=ax,legend=False)
        ax.set_title(str(windt)+' Days Draw Down')
        #pyplot.show()
        canvas = FigureCanvasTkAgg(fig, master=windowt)
        canvas.get_tk_widget().pack()
        
    def check_factor(self):
        if self.factor.empty:
            messagebox.showinfo("MessageBox", "Please upload or download factors data by button [13] or [14]!")
            return        
        windowt = tk.Toplevel()
        #windowt.geometry("300x300")
        tk.Label(windowt, text='Upload or download factors namelist: ',font=('Arial',13,'bold')).pack()
        tk.Label(windowt, text=list(self.factor)).pack()

In [23]:
pa = PortfolioAnalyzer()

window = tk.Tk()
window.title('Portfolio Analyzer')
window.geometry('600x600')
i=30 #place
j=20 #place
k=100

w = tk.Label(window, text="Active Portfolio Analyzer",font=('Arial',20,'bold'))
w.pack()
w1 = tk.Label(window, text="Instruction: * must do. More Details",compound=LEFT,font=('Arial',12))
w1.place(x=10,y=30)
###################### PART 1#######################
w2 = tk.Label(window, text="Part1 Initialization:",compound=LEFT,font=('Arial',13,'bold'))
w2.place(x=10,y=50)
w2 = tk.Label(window, text="Please select one of following methods [1] or [2] to initialization*:",compound=LEFT,font=('Arial',12))
w2.place(x=10,y=70)

li=tk.Button(window,text='[1] Upload Pool Information',font=('Arial',12),width=30,height=2,command=pa.initialization) 
li.place(x=10,y=92)
li=tk.Button(window,text='[2] Upload Fund NAV',font=('Arial',12),width=30,height=2,command=pa.initialization_method2) 
li.place(x=310,y=92)

w3 = tk.Label(window, text="If use button [1], please select one of following method to compute fund NAV*:",compound=LEFT,font=('Arial',12))
w3.place(x=10,y=125)

lx=tk.Button(window,text='[3] By Analyst',font=('Arial',12),width=15,height=2,command=pa.analyst) 
lx.place(x=10,y=147)
ly=tk.Button(window,text='[4] By Sector',font=('Arial',12),width=15,height=2,command=pa.sector) 
ly.place(x=160,y=147)
lz=tk.Button(window,text='[5] By Theme',font=('Arial',12),width=15,height=2,command=pa.theme) 
lz.place(x=310,y=147)
lz=tk.Button(window,text='[6] ALL',font=('Arial',12),width=15,height=2,command=pa.allstock) 
lz.place(x=460,y=147)

###################### PART 2#######################
w4 = tk.Label(window, text="Part2: Funds Performance:",compound=LEFT,font=('Arial',13,'bold'))
w4.place(x=10,y=200)
w41 = tk.Label(window, text="Fund performance summary:",compound=LEFT,font=('Arial',12))
w41.place(x=10,y=220)
lp1=tk.Button(window,text='[7] Plot Performance',font=('Arial',12),width=15,height=2,command=pa.plot_performance) 
lp1.place(x=10,y=242)
lp2=tk.Button(window,text='[8] Indicators',font=('Arial',12),width=15,height=2,command=pa.indicator) 
lp2.place(x=160,y=242)
w42 = tk.Label(window, text="Fund risk analysis: Please enter risk [11] before [12]:",compound=LEFT,font=('Arial',12))
w42.place(x=10,y=275)
lp3=tk.Button(window,text='[11] Enter Risk',font=('Arial',12),width=15,height=2,command=pa.enter_risk_budget) 
lp3.place(x=10,y=297)
lp4=tk.Button(window,text='[12] Risk Budgets',font=('Arial',12),width=15,height=2,command=pa.risk_parity)
lp4.place(x=160,y=297)
w43 = tk.Label(window, text="Fund performance VS market:",compound=LEFT,font=('Arial',12))
w43.place(x=310,y=220)
lp5=tk.Button(window,text='[9] Draw Down',font=('Arial',12),width=15,height=2,command=pa.draw_down)
lp5.place(x=310,y=242)
lp6=tk.Button(window,text='[10] Market Neutral',font=('Arial',12),width=15,height=2,command = pa.market_neutral)
lp6.place(x=460,y=242)

###################### PART 3#######################
w5 = tk.Label(window, text="Part3: Factors Analysis:",compound=LEFT,font=('Arial',13,'bold'))
w5.place(x=10,y=330)
w6 = tk.Label(window, text="Please select one of following methods [13] or [14] to get factors data before using functions in part3*:",compound=LEFT,font=('Arial',12))
w6.place(x=10,y=350)
lf=tk.Button(window,text='[13] Upload Factors Data',font=('Arial',12),width=30,height=2,command=pa.upload_factor) 
lf.place(x=10,y=372)
lf=tk.Button(window,text='[14] Download Factors Data',font=('Arial',12),width=30,height=2,command=pa.download_factor) 
lf.place(x=310,y=372)
w7 = tk.Label(window, text="Choose factors by LASSO or Stepwise OLS:",compound=LEFT,font=('Arial',12))
w7.place(x=10,y=405)
lf1=tk.Button(window,text='[15] LASSO',font=('Arial',12),width=15,height=2,command=pa.LASSO_selected) 
lf1.place(x=10,y=427)
lf2=tk.Button(window,text='[16] Step-wise OLS',font=('Arial',12),width=15,height=2,command=pa.forward_selected) 
lf2.place(x=160,y=427)
w8 = tk.Label(window, text="Get Alpha & Beta by OLS or Kalman Filter:",compound=LEFT,font=('Arial',12))
w8.place(x=310,y=405)
lf3=tk.Button(window,text='[17] OLS',font=('Arial',12),width=15,height=2,command=pa.OLS_method) 
lf3.place(x=310,y=427)
lf4=tk.Button(window,text='[18] Kalman Filter',font=('Arial',12),width=15,height=2,command=pa.KF_method) 
lf4.place(x=460,y=427)
w9 = tk.Label(window, text="Factor timing or neutral analysis:",compound=LEFT,font=('Arial',12))
w9.place(x=10,y=460)
lf5=tk.Button(window,text='[19] Factor Timing',font=('Arial',12),width=15,height=2,command=pa.factor_timing) 
lf5.place(x=10,y=482)
lf7=tk.Button(window,text='[20] Factor Neutral',font=('Arial',12),width=15,height=2,command=pa.factor_neutral) 
lf7.place(x=160,y=482)
wf1 = tk.Label(window, text="PCA analysis:",compound=LEFT,font=('Arial',12))
wf1.place(x=310,y=460)
lf6=tk.Button(window,text='[21] PCA',font=('Arial',12),width=15,height=2,command=pa.PCA) 
lf6.place(x=310,y=482)
wf1 = tk.Label(window, text="Check factors name:",compound=LEFT,font=('Arial',12))
wf1.place(x=310,y=460)
lf6=tk.Button(window,text='[22] Check Factors',font=('Arial',12),width=15,height=2,command=pa.check_factor) 
lf6.place(x=460,y=482)

###################### PART 4#######################
ww = tk.Label(window, text="Part4: Weighted Analysis:",compound=LEFT,font=('Arial',13,'bold'))
ww.place(x=10,y=515)
ww1 = tk.Label(window, text="If upload pool information, compute min/max weighted:",compound=LEFT,font=('Arial',12))
ww1.place(x=10,y=535)
lw=tk.Button(window,text='[23] Min/Max Weighted',font=('Arial',12),width=30,height=2,command=pa.min_max) 
lw.place(x=10,y=557)
#canvas=tk.Canvas(window,height=400,width=400)
#canvas.create_line(40,55,150,55)
#### 
k=100
#canvas.create_line(40,55+k,150,55+k)
#canvas.pack()

window.mainloop()